In [11]:
%pip install scikit-learn
%pip install keybert

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


  Using cached keybert-0.8.4.tar.gz (29 kB)
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Using cached sentence_transformers-2.6.1-py3-none-any.whl (163 kB)
  Using cached rich-13.7.1-py3-none-any.whl (240 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl (87 kB)
  Using cached transformers-4.39.3-py3-none-any.whl (8.8 MB)
                                              0.0/198.6 MB ? eta -:--:--
                                              0.0/198.6 MB ? eta -:--:--
                                            0.0/198.6 MB 325.1 kB/s eta 0:10:11
                                            0.0/198.6 MB 279.3 kB/s eta 0:11:51
                                            0.0/198.6 MB 279.3 kB/s eta 0:11


[notice] A new release of pip is available: 23.1.2 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


     -----------------                      92.9/198.6 MB 46.5 kB/s eta 0:37:53
     -----------------                      93.0/198.6 MB 46.8 kB/s eta 0:37:39
     -----------------                      93.0/198.6 MB 46.8 kB/s eta 0:37:39
     -----------------                      93.0/198.6 MB 46.2 kB/s eta 0:38:09
     -----------------                      93.0/198.6 MB 46.2 kB/s eta 0:38:09
     -----------------                      93.0/198.6 MB 46.2 kB/s eta 0:38:08
     -----------------                      93.0/198.6 MB 46.2 kB/s eta 0:38:08
     -----------------                      93.0/198.6 MB 46.1 kB/s eta 0:38:09
     -----------------                      93.0/198.6 MB 46.1 kB/s eta 0:38:09
     -----------------                      93.0/198.6 MB 46.1 kB/s eta 0:38:09
     -----------------                      93.0/198.6 MB 46.1 kB/s eta 0:38:10
     -----------------                      93.0/198.6 MB 46.1 kB/s eta 0:38:10
     -----------------                  

In [12]:
import pandas as pd
from keybert import KeyBERT
import matplotlib.pyplot as plt
from collections import Counter

c:\Users\dell\AppData\Local\Programs\Python\Python311\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [13]:
rating_df=pd.read_csv("../data/rating.csv")
domain_locations_df = pd.read_csv("../data/domains_location.csv")
traffic_data_df = pd.read_csv("../data/traffic.csv")

# Drop duplicates
rating_df.drop_duplicates(inplace=True)
domain_locations_df.drop_duplicates(inplace=True)
traffic_data_df.drop_duplicates(inplace=True)

# Drop unnecessary columns
if 'source_id' in rating_df.columns:
    rating_df.drop(columns=['source_id'], inplace=True)

if 'url_to_image' in rating_df.columns:
    rating_df.drop(columns=['url_to_image'], inplace=True)

# Drop rows with missing values
rating_df = rating_df.dropna(axis=0)
domain_locations_df = domain_locations_df.dropna(axis=0)

# rating_df['domain'] = rating_df['url'].apply(get_domain)

merge_df=pd.merge(rating_df, domain_locations_df ,left_on='source_name', right_on ='SourceCommonName' ,how ='left')
merge_df=pd.merge(merge_df, traffic_data_df , left_on ='source_name' ,right_on='Domain' ,how ='left')

In [15]:

model = KeyBERT('distilbert-base-nli-mean-tokens')

def extract_keywords(text):
    return model.extract_keywords(text, keyphrase_ngram_range=(1, 1), stop_words='english', top_n=5)

merge_df['title_keywords'] = merge_df['title'].apply(extract_keywords)
merge_df['content_keywords'] = merge_df['content'].apply(extract_keywords)

all_title_keywords = [keyword for keywords in merge_df['title_keywords'] for keyword in keywords]
all_content_keywords = [keyword for keywords in merge_df['content_keywords'] for keyword in keywords]

title_keyword_counts = Counter(all_title_keywords)
content_keyword_counts = Counter(all_content_keywords)

sorted_title_keywords = sorted(title_keyword_counts.items(), key=lambda x: x[1], reverse=True)
sorted_content_keywords = sorted(content_keyword_counts.items(), key=lambda x: x[1], reverse=True)

# Plot the top 10 keywords extracted from titles
plt.figure(figsize=(10, 5))
plt.bar(*zip(*sorted_title_keywords[:10]))
plt.xlabel('Keyword')
plt.ylabel('Frequency')
plt.title('Top 10 Keywords Extracted from Titles')
plt.xticks(rotation=45)
plt.show()

# Plot the top 10 keywords extracted from content
plt.figure(figsize=(10, 5))
plt.bar(*zip(*sorted_content_keywords[:10]))
plt.xlabel('Keyword')
plt.ylabel('Frequency')
plt.title('Top 10 Keywords Extracted from Content')
plt.xticks(rotation=45)
plt.show()
